
## Curso *Ingeniería de características*

# Detección de anomalías 

**Julio Waissman Vilanova**

<table align="center">
      <td align="center"><a target="_blank" href="https://www.unison.mx">
            <img src="https://www.unison.mx/wp-content/themes/awaken/images/logo.png"  height="70px" style="padding-bottom:5px;"  /></a></td>  
      <td align="center"><a target="_blank" href="https://www.gob.mx/cenace">
            <img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" height="70px" style="padding-bottom:5px;" /></a></td>
      <td align="center"><a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/anomalias/taller_solar.ipynb">
            <img src="https://i.ibb.co/2P3SLwK/colab.png"  height="70px" style="padding-bottom:5px;" /></a></td>

</table>


In [ ]:
!pip install openpyxl
!pip install scikit-learn
!pip install hvplot
!pip install -U bokeh

In [ ]:
import numpy as np

import holoviews as hv
import hvplot
import hvplot.pandas 

import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 7]

## Cargando los datos

In [ ]:
url = "https://github.com/mcd-unison/ing-caract/raw/main/ejemplos/anomalias/Dataset_GeneracionFV.xlsx"
df = pd.read_excel(url, sheet_name=1)
df.info()

In [ ]:
df.rename(
    columns={"Generacion_FV_MW": "Generacion", "Radiacion_FV_W/m2": "Radiacion"}, 
    inplace=True
)
df.index = df.Fecha
df['Hora'] = df.index.hour
df['Dia'] = df.index.dayofweek
df['Mes'] = df.index.month
df

## Análisis exploratorio de datos


1. ¿Cual es la radiación y generación promedio por hora?
2. ¿Cual es la radiación y generación promedio por mes?
3. ¿Existe una correlación entre radiación y generación? ¿Es lineal?
4. ¿A que horas inicia la generación por mes?

In [ ]:
%env HV_DOC_HTML=true
hv.extension('bokeh')

df.hvplot.scatter(
    x='Radiacion',
    y='Generacion',
    #by='Dia',
    #groupby='Mes',
    title="Radiación vs generación de energía"
)

¿Como se comporta la generación respecto a la hora del día? ¿Y respecto al mes? ¿Cómo se ve a lo largo de un día? ¿El día de la semana (o del més) tendría algo que ver? Agrega las celdas de código acomañadas de las celdas de comentarios que sean necesarios para un análisis preliminar de datos.

In [ ]:
%env HV_DOC_HTML=true
hv.extension('bokeh')

# Agrega cuantas celdas y comentarios consideres necesarios
df.hvplot.box(
    y="Generacion", 
    by="Hora", 
    #groupby='Mes',
    title="Distribución de la generación por hora del día"
)

In [ ]:
%env HV_DOC_HTML=true
import hvplot
hv.extension('bokeh')

df2 = df.copy()
#df2.reset_index(drop=True, inplace=True)
df2.drop(columns=['Fecha'], inplace=True)

explorer = hvplot.explorer(df2)
explorer

In [ ]:
# Agrega cuantas celdas creas necesarias

¿Cres que existen outiers que se pueden analizar por variables en forma independiente? ¿Cuales? ¿Hay algun caso que revisar con los propietarios del proceso?

## Detección de anomalías

Vamos a probar varios métodos de detección de anomalías para compararlos entre ellos. Para esto vamos a utilizar los métodos clásicos que vienen en `sci-kit learn` los cuales son:

- Ajuste de una distribución (elíptica)
- Detección de *outliers* por bosques de aislamiento
- Detección de *outliers* por *Local Outlier Factor* (LOF)

Vamos a usar un caso muy simple de ejemplo (solo Radiación versis Generación) y se deja a los participantes que prueben un proceso multivariado.

In [ ]:
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

X = df[["Radiacion", "Generacion"]].to_numpy()
X.shape

#### Empezamos con las envolventes elípticas

In [ ]:
# Estimador robusto (MCD)
robust_cov = MinCovDet().fit(X)
estimado_MCD = robust_cov.mahalanobis(X)

# Estimador empírico (MLE)
emp_cov = EmpiricalCovariance().fit(X)
estimado_MLE = emp_cov.mahalanobis(X)

print('Matrices de covarianza estimadas:\n' +
      f'MCD:\n{robust_cov.covariance_}\n' +
      f'MLE:\n{emp_cov.covariance_}')

In [ ]:
plt.scatter(x=X[:,0], y=X[:,1], c=estimado_MCD, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
plt.scatter(x=X[:,0], y=X[:,1], c=np.where(estimado_MCD < 20000, 0, 10), cmap='jet')
plt.show()

In [ ]:
plt.scatter(x=X[:,0], y=X[:,1], c=estimado_MLE, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
plt.scatter(x=X[:,0], y=X[:,1], c=np.where(estimado_MLE < 5, 0, 10), cmap='jet')
plt.show()

#### Bosques de aislamiento

In [ ]:
clf = IsolationForest(n_estimators=100, warm_start=True)
clf.fit(X)

# Si warm_start es True, se le puede ir agregando estimadores poco a poco
#clf.set_params(n_estimators=20) 
#clf.fit(X)

estimado_IF = clf.decision_function(X)

plt.scatter(x=X[:,0], y=X[:,1], c=estimado_IF, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
plt.scatter(x=X[:,0], y=X[:,1], c=np.where(estimado_IF > -0.1, 1, 0), cmap='jet')
plt.show()

#### Local Outlier Factor

In [ ]:
LocalOutlierFactor?

In [ ]:
clf = LocalOutlierFactor(n_neighbors=100)
estimado_LOF = clf.fit_predict(X)

plt.scatter(x=X[:,0], y=X[:,1], c=estimado_LOF, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
plt.scatter(x=X[:,0], y=X[:,1], c=np.where(estimado_LOF > 0.0, 1, 0), cmap='jet')
plt.show()

¿Qué variables habría que integrar en el estudio? ¿Que pasaría si agregamos la hora para la detección de *outliers*? ¿Cambiaría mucho los puntos considerados como outliers? 